In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
#from sklearn.metrics import r2_score
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')
print(device)
%matplotlib inline
np.random.seed(1)
plt.style.use("seaborn-talk") #"classic" "seaborn-talk" "seaborn"
path_to_h5 =  "/home/leonov/Baikal/Cut_8_nu/APRIL/Ordered/Data/mc_baikal_norm_cut-8_ordered_equal_big.h5"
#Baikal/Cut_8_nu/APRIL/Ordered/Data
# можно отделить хвост и отдельно прогнать его по другой сетке
def make_trainset_noxyz(i, di = 1, tr_set_len = 128,Batch_size = 64):
    with h5py.File(path_to_h5, 'r') as hf:
            Data = hf['train/data/'][i*int(tr_set_len) : (i+di)*int(tr_set_len),:32]
            Polar=hf["/train/ev_chars"][i*int(tr_set_len) : (i+di)*int(tr_set_len),0]*(np.pi)/180
            Azimut=hf["/train/ev_chars"][i*int(tr_set_len) : (i+di)*int(tr_set_len),1]*(np.pi)/180
            x=np.expand_dims(np.sin(Polar)*np.cos(Azimut),axis=1)
            y=np.expand_dims(np.sin(Polar)*np.sin(Azimut),axis=1)
            z=np.expand_dims(np.cos(Polar),axis=1)
            target=torch.FloatTensor(np.concatenate((x,y,z) ,axis=1))
            Data = torch.FloatTensor(Data.swapaxes(1, -1)) # надо,т.к. второй индекс должен быть количеством   последовательностей
            trainDataset = torch.utils.data.TensorDataset(Data, target)
            trainLoader = torch.utils.data.DataLoader(dataset = trainDataset, batch_size=Batch_size) #,sampler = sampler    
    return  trainLoader
def make_valset_noxyz(i=0, di = 1, tr_set_len = 1000, Batch_size = 64):
    with h5py.File(path_to_h5, 'r') as hf:
            Data = hf['val/data/'][i*int(tr_set_len) : (i+di)*int(tr_set_len),:32] # ибо очень редко как то задействовано более чем 32 OM
            Data = torch.FloatTensor(Data.swapaxes(1, -1)) 
            Polar=hf["/val/ev_chars"][i*int(tr_set_len) : (i+di)*int(tr_set_len),0]*(np.pi)/180
            Azimut=hf["/val/ev_chars"][i*int(tr_set_len) : (i+di)*int(tr_set_len),1]*(np.pi)/180
            #
            x=np.expand_dims(np.sin(Polar)*np.cos(Azimut),axis=1)
            y=np.expand_dims(np.sin(Polar)*np.sin(Azimut),axis=1)
            z=np.expand_dims(np.cos(Polar),axis=1)
            target=torch.FloatTensor(np.concatenate((x,y,z) ,axis=1))
            testDataset = torch.utils.data.TensorDataset(Data, target)
            testLoader = torch.utils.data.DataLoader(dataset = testDataset, batch_size=Batch_size)
    return testLoader
# вектора в углы
def v_to_angles( Predicted, Real,
                p_hist, az_hist, p_error_hist,
                to_hists = True): 
    #Predict = torch.nn.functional.normalize(model(torch.Tensor(Predict).to(device).float() )).cpu().detach().numpy()
    v_pred = torch.nn.functional.normalize(Predicted.detach()) # нормализую
    #полярный получаю просто и в градусах
    polar_real = torch.acos(Real[:,-1])/(np.pi)*180 
    polar_predicted = torch.acos(v_pred[:,-1])/(np.pi)*180 
    # azimut
    azimut =torch.acos( v_pred[:,0]/((v_pred[:,0])**2+(v_pred[:,1])**2+1e-8)**0.5 ) # добавляю  в знаменатель добавку чтоб наны не получить
    azimut = azimut+(torch.sign(v_pred[:,1])**2)*(1-torch.sign(v_pred[:,1]))*(np.pi-azimut) # torch.sign(v[:,1])**2 на случай нулевых углов,которых нет
    azimut = azimut/(np.pi)*180 # привожу к градусам
    if not to_hists: 
        return polar,azimut
    azimut=azimut.short()
    #polar = polar.short()
    for pol_pred, pol_real, az in zip(polar_predicted, polar_real, azimut):
        p_error_hist[round(abs((pol_pred-pol_real).item()),1)] += 1
        p_hist[pol_pred.short().item()] += 1
        az_hist[az.item()] += 1
#{k:0 for k in np.arange(0.0, 180, 0.1)}     
def resolution_calculation(v1,v2,hist,to_hist = True):
    v1, v2 = v1.double(), v2.double()    
    if not to_hist:
        return torch.acos(torch.nn.functional.cosine_similarity(v1,v2,dim = 1))/np.pi*180
    res = torch.acos(torch.nn.functional.cosine_similarity(v1,v2,dim = 1))/np.pi*180
    res = torch.nn.functional.cosine_similarity(v1,v2,dim = 1)
    res = torch.acos(res)/np.pi*180
    for res_value in res:
        try:
                hist[round(res_value.item(),1)] += 1
        except KeyError:
            hist[0.0] += 1
            print(res_value.item(),"torch is bad at acos calculation")
            #print(res_value)
            #print("v1 = " ,v1, v1.shape ,"v2 = ", v2, v2.shape,
            #      "res_value = ",res_value, res_value.shape,"res = ",
            #      res,res.shape,"res_value_round = ", round(res_value.item(),1), sep= "\n")
            #print(torch.nn.functional.cosine_similarity(v1,v2,dim = 1))
            #raise KeyError
def loss_plot(list_test, list_train , path , save = True):
    plt.figure(figsize=(9,6))
    plt.plot(np.arange(len(list_test)), list_test, label='val', linewidth=2)
    plt.plot(np.arange(len(list_train)), list_train, label='train', linewidth=2)
    plt.title('Loss_plot')
    plt.xlabel('iterations')
    plt.ylabel('Loss')
    plt.legend()
    if save == True:
        plt.savefig(path)
    plt.show()
def res_plot(train_dict,val_dict, path = None,save = True, res_or_polar = "Resolution " ,size= 13):
    plt.figure(figsize=(13,5))
    colours=["red","red"]
    names = ["train" , "val"]
    for i, d in enumerate([train_dict,val_dict]):
        plt.subplot(1,2,i+1)
        s = sum(d.values())
        prep_inter_s ,inter_s = 0, 0
        res_50, res_68 =0,0
        for key in list(d.keys()):
            if inter_s/s >= 0.68:
                alpha = (inter_s-0.68*s)/(inter_s-prep_inter_s) # alpha*inter_s+(1-alpha)*prep_inter_s == 0.68*s
                res_68 =round(key + 0.1*alpha,2)  #alpha*(key-0.1)+(1-alpha)*(key)
                break
            if inter_s/s >= 0.5 and res_50 == 0:
                alpha = (inter_s-0.5*s)/(inter_s-prep_inter_s)
                res_50 =round(key + 0.1*alpha,2)
            prep_inter_s = inter_s 
            inter_s  += d[key]  
        a=plt.step(list(d.keys())[:300], list(d.values())[:300], color=colours[i],alpha = 0.6)
        plt.bar(res_50, max(d.values()), width=0.5,label=names[i] + "50%" + res_or_polar +"= "+str(res_50),
                color="yellow" , alpha =0.7 )
        plt.bar(res_68, max(d.values()), width=0.5,label=names[i] + "68%" + res_or_polar +" = "+str(res_68) ,
                color="orange",alpha = 0.7)
        plt.legend()
        plt.xlabel(res_or_polar + "in_grad",fontsize= size); plt.title(res_or_polar + names[i],fontsize= size)
    if save == True:
        plt.savefig(path) 
    plt.show()

def angle_hist(hist_polar,hist_azimut,path, save =True,size= 13,name = "train"):
    with h5py.File(path_to_h5, 'r') as hf:
        plt.figure(figsize= (13,10))
        sum_value = sum(hist_polar.values())
        Polar=hf["/"+name+"/ev_chars"][ : ,0] 
        Azimut=hf["/"+name+"/ev_chars"][ : ,1]
        plt.subplot(2,1,1)
        plt.hist(Polar,bins=180,label=name+" Polar",density=True,histtype="step",color="blue")
        plt.bar(list(hist_polar.keys())[:100], np.array(list(hist_polar.values())[:100])/sum_value, 
                color="red",label=name+" Polar_Predicted")
        plt.legend()
        plt.xlabel("Polar_Angle",fontsize= size); plt.title(name+"_Polar",fontsize= size)
        plt.subplot(2,1,2) 
        plt.hist(Azimut,bins=360,label=name+" Azimut",density=True,histtype="step",color="blue")
        plt.bar(list(hist_azimut.keys()), np.array(list(hist_azimut.values()))/sum_value,
                    align = 'center', color="red",label=name+" Azimut_Predicted")
        plt.legend()
        plt.xlabel("Azimut_Angle",fontsize= size) ;plt.title(name+"_Azimut",fontsize= size)
        if save == True:
            plt.savefig(path) 
        plt.show()


cuda:0


In [ ]:
tr_set_len = 512*100
seq = [j for j in range(int( 1881011/tr_set_len))]
print('Num of sub-epochs in Epoch = ', len(seq), '\n')
len_seq = len(seq)

#Определяем модель
#model = Net.to(device)
def fitting(model, scheduler_Exp, scheduler_MultiStep , optimizer,
        epochs_num = 25, batch_size = 64,
        criterion=torch.nn.L1Loss(),
        save_weights = True, save_plot = True, save_resolution = True,  save_angles = True, save_polar_error = True,   
        suffix = "Nu_MAE_Res_1D",
        path_begin = "/home/leonov/Baikal/Cut_8_nu/APRIL/Ordered"
           ):
    #optimizer = torch.optim.Adam(model.parameters(),lr=learn_rate)
    #scheduler_Exp = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=1)
    #scheduler_MultiStep = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[10,20,30,40,42], gamma=0.5) 
    loss_train = []
    loss_test = []
    hist_train_polar_error = {round(k,1):0 for k in np.arange(0.0, 180, 0.1)} # здесь будут polar error всех событий
    hist_val_polar_error = {round(k,1):0 for k in np.arange(0.0, 180, 0.1)}
    hist_train_res = {round(k,1):0 for k in np.arange(0.0, 180, 0.1)} # здесь будут гистограммы  разрешения всех событий
    hist_val_res = {round(k,1):0 for k in np.arange(0.0, 180, 0.1)}    
    hist_train_polar = {k:0 for k in np.arange(0, 181, 1)} # гистограммы  предсказанных углов для сравнения с реальным распределением 
    hist_train_azimut = {k:0 for k in np.arange(0, 361, 1)}
    hist_val_polar = {k:0 for k in np.arange(0, 181, 1)} 
    hist_val_azimut = {k:0 for k in np.arange(0, 361, 1)}    
    num = 0
    for n in range(1, epochs_num+1):
        #training
        print('\n','Indeed Epoch = ', n)
        for i in seq:
            train_Loader = make_trainset_noxyz(i,1,tr_set_len,Batch_size = batch_size)
            for x_batch,y_batch in train_Loader:
                optimizer.zero_grad()
                outp = model(x_batch.to(device).float())
                loss =   criterion(outp,y_batch.to(device).float())
                loss.backward()
                optimizer.step()
                # полученный вектор направления превращаю в углы и добавляю в гистограммы
                if n == epochs_num:
                    v_to_angles(Predicted = outp, Real = y_batch.to(device), p_error_hist = hist_train_polar_error,
                                p_hist = hist_train_polar, az_hist = hist_train_azimut, to_hists = True)
                    resolution_calculation(outp,y_batch.to(device),hist = hist_train_res,to_hist = True)
            if (num%(len_seq//3) == 0):
                rand_ind=np.random.randint(0,15)
                print('Sub-epoch number = ', num)
                loss_train.append(loss.item())
                model.eval()
                testLoader = make_valset_noxyz(rand_ind,1,500, Batch_size = batch_size)
                test_loss=0
                count=0
                for x_test_batch,y_test_batch in testLoader:
                    outp = model(x_test_batch.to(device).float())
                    test_loss +=  criterion(outp,y_test_batch.to(device).float()).item()
                    count+=1
                test_loss /=count
                loss_test.append(test_loss)
                model.train()
                print("train_loss = ",loss.item(),"  val_loss = ",test_loss)
            num+=1
        scheduler_Exp.step()
        scheduler_MultiStep.step()
        
    model.eval()
    FinalLoader = make_valset_noxyz(0,-1,1, Batch_size = batch_size) # делаю  loader из всего датасета
    for x_test_batch,y_test_batch in FinalLoader:
        outp = model(x_test_batch.to(device).float())
        v_to_angles(Predicted = outp, Real = y_test_batch.to(device), p_error_hist = hist_val_polar_error,
                    to_hists = True,p_hist = hist_val_polar, az_hist = hist_val_azimut)
        resolution_calculation(outp, y_test_batch.to(device), hist = hist_val_res, to_hist = True)
        
    if save_weights == True:
        torch.save(model.state_dict(), path_begin + "/states/" + suffix + "model")
        torch.save(optimizer.state_dict(), path_begin + "/states/" + suffix + "opt")
    # график лосса
    loss_plot(loss_test, loss_train , path_begin + "/Images/Loss/" + suffix + "LOSS.png", 
              save_plot )
    #  гистограмма ошибок  полярного угла
    res_plot(hist_train_polar_error, hist_val_polar_error, 
             path = path_begin + "/Images/Polar_Error/" + suffix+ "Polar_Error.png",
             save = save_polar_error, res_or_polar = "Polar_Error ")    
    # гистограммы разрешения
    res_plot(hist_train_res,hist_val_res,
             path = path_begin + "/Images/Resolution/" + suffix + "RESOLUTIONS.png",
             save = save_resolution,res_or_polar = "Resolution ")
    #гистограммы углов
    angle_hist(hist_train_polar,hist_train_azimut, name='train',
               path = path_begin + "/Images/Angles/" + suffix + "Angles_train.png", 
               save = save_angles)
    angle_hist(hist_val_polar,hist_val_azimut, name='val',
               path = path_begin + "/Images/Angles/" + suffix + "Angles_val.png", 
               save = save_angles)
    loss_lists = [loss_train , loss_test]
    polar_hists = [hist_train_polar , hist_val_polar]
    azimut_hists = [hist_train_azimut, hist_val_azimut]
    res_hists = [hist_train_res ,hist_val_res]
    polar_error = [hist_train_polar_error, hist_val_polar_error]
    model.train()
    return  loss_lists, polar_hists, azimut_hists ,res_hists ,polar_error




In [14]:
class ResNet_Block(torch.nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.module = torch.nn.Sequential(    
            torch.nn.Conv1d(input_size, input_size*2,  kernel_size=3, stride=1,padding=1),
            torch.nn.BatchNorm1d(input_size*2),
            torch.nn.PReLU(),
            torch.nn.Conv1d(input_size*2, input_size*2,  kernel_size=3, stride=1,padding=1),
            torch.nn.BatchNorm1d(input_size*2),
            torch.nn.PReLU(),
            torch.nn.Conv1d(input_size*2,input_size,  kernel_size=3, stride= 1 ,padding=1),
            torch.nn.BatchNorm1d(input_size),
            torch.nn.PReLU()
          )
        self.conv = torch.nn.Sequential( 
            torch.nn.Conv1d(input_size, input_size, kernel_size =1),
            torch.nn.PReLU()
        )
    def forward(self, inputs):
          return  (self.module(inputs) +self.conv(inputs))

net = torch.nn.Sequential(    
    torch.nn.Conv1d(5, 5,  kernel_size= 3 , stride = 1 ,padding= 1 ),
    torch.nn.BatchNorm1d(5),
    torch.nn.PReLU(),
    ResNet_Block(5),
    Transition_Block(5), #16
    ResNet_Block(10), #16
    Transition_Block(10), #8
    ResNet_Block(20),
    Transition_Block(20), #4
    ResNet_Block(40),  
    torch.nn.Flatten(),
    
)
a = torch.ones((512,5 ,32))
net(a).shape

torch.Size([512, 160])

In [19]:
class ResNet_Block(torch.nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.module = torch.nn.Sequential(    
            torch.nn.Conv1d(input_size, input_size*2,  kernel_size=3, stride=1,padding=1),
            torch.nn.BatchNorm1d(input_size*2),
            torch.nn.PReLU(),
            torch.nn.Conv1d(input_size*2, input_size*2,  kernel_size=3, stride=1,padding=1),
            torch.nn.BatchNorm1d(input_size*2),
            torch.nn.PReLU(),
            torch.nn.Conv1d(input_size*2,input_size,  kernel_size=3, stride= 1 ,padding=1),
            torch.nn.BatchNorm1d(input_size),
            torch.nn.PReLU()
          )
        self.conv = torch.nn.Sequential( 
            torch.nn.Conv1d(input_size, input_size, kernel_size =1),
            torch.nn.PReLU()
        )
    def forward(self, inputs):
          return  (self.module(inputs) +self.conv(inputs))
class Transition_Block(torch.nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.trans_module = torch.nn.Sequential(
            torch.nn.Conv1d(input_size, input_size*2, kernel_size=3, stride=2,padding=1 ),
            torch.nn.BatchNorm1d(num_features=input_size*2),
            torch.nn.PReLU()
        )
    def forward(self, inputs):
        return  self.trans_module(inputs) 
    

a = torch.ones((512,5 ,32))
net = torch.nn.Sequential(    
    torch.nn.Conv1d(5, 5,  kernel_size= 3 , stride = 1 ,padding= 1 ),
    torch.nn.BatchNorm1d(5),
    torch.nn.PReLU(),
    ResNet_Block(5), #32
    Transition_Block(5), #16
    ResNet_Block(10), #16
    Transition_Block(10), #8
    ResNet_Block(20),
    Transition_Block(20), #4
    ResNet_Block(40), # 4
    torch.nn.Flatten(),
    torch.nn.Linear(160,40),
    torch.nn.BatchNorm1d(40),
    torch.nn.PReLU(),
    torch.nn.Linear(40,20),
    torch.nn.BatchNorm1d(20),
    torch.nn.PReLU(),
    torch.nn.Linear(20 ,3)
) 


#net(a).shape
sum(p.numel() for p in net.parameters()   )

65240

In [ ]:
model = net.to(device)
learn_rate =6e-3
opt = torch.optim.Adam(model.parameters(),lr=learn_rate)
sch_Exp = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.99)
sch_MultiStep = torch.optim.lr_scheduler.MultiStepLR(opt,milestones=[10,20,30,40,45], gamma=0.7) 
loss_lists, polar_hists, azimut_hists ,res_hists ,polar_error =fitting(model,sch_Exp , sch_MultiStep, opt,
                                                                       suffix = "Nu_MAE_Res_1D",
                                                                       epochs_num=10, batch_size =64)